In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("IcebergNessieExample").getOrCreate()

25/03/30 19:36:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sparkContext.getConf().getAll()

[('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog'),
 ('spark.sql.catalog.nessie.uri', 'http://nessie:19120/api/v1'),
 ('spark.hadoop.fs.s3a.path.style.access', 'true'),
 ('spark.sql.catalog.nessie.ref', 'main'),
 ('spark.sql.warehouse.dir',
  'file:/home/iceberg/notebooks/notebooks/spark-warehouse'),
 ('spark.app.initial.jar.urls',
  'spark://35fa2fe31596:44111/jars/org.projectnessie.nessie-integrations_nessie-spark-extensions-3.5_2.12-0.103.2.jar,spark://35fa2fe31596:44111/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.8.1.jar'),
 ('spark.hadoop.fs.s3a.access.key', 'minioadmin'),
 ('spark.sql.catalog.nessie.s3.path-style-access', 'true'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.extensions',
  'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions'),
 ('spark.app.id', 'local-1743363403156'),

In [3]:
spark.sql("use nessie")

25/03/23 21:11:15 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/03/23 21:11:15 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/03/23 21:11:16 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/03/23 21:11:16 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@10.89.0.5
25/03/23 21:11:16 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [4]:
spark.sql("CREATE BRANCH te")

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `default` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [12]:
spark.sql("CREATE TABLE IF NOT EXISTS default.employees (id INT, name STRING, salary DOUBLE) using iceberg")
spark.sql("INSERT INTO default.employees VALUES (1, 'Alice', 75000), (2, 'Bob', 80000)")

DataFrame[]

In [5]:
spark.sql('show namespaces').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [14]:
spark.sql("CREATE TABLE IF NOT EXISTS nessie.employees (id INT, name STRING, salary DOUBLE)")

spark.sql("INSERT INTO default.employees VALUES (1, 'Alice', 75000), (2, 'Bob', 80000)")

df = spark.sql("SELECT * FROM default.employees")
df.show()

+---+-----+-------+
| id| name| salary|
+---+-----+-------+
|  1|Alice|75000.0|
|  1|Alice|75000.0|
|  1|Alice|75000.0|
|  2|  Bob|80000.0|
|  2|  Bob|80000.0|
|  2|  Bob|80000.0|
|  1|Alice|75000.0|
|  2|  Bob|80000.0|
+---+-----+-------+



In [9]:
df = spark.sql("SELECT * FROM default.employees")
df.show()

+---+-----+-------+
| id| name| salary|
+---+-----+-------+
|  1|Alice|75000.0|
|  2|  Bob|80000.0|
|  1|Alice|75000.0|
|  2|  Bob|80000.0|
+---+-----+-------+



In [20]:
spark.sql('create namespace default')

DataFrame[]